In [31]:
import json
import nltk
#nltk.download('punkt_tab')
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np
import random
import pandas as pd
import warnings
#!pip install tensorflow
import tensorflow.keras.models
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, Activation, Input, Flatten
from tensorflow.keras.models import Sequential

In [3]:
try:
    path = 'C:/Users/61450/Desktop/New folder/Chatbot/intents.json'
except FileNotFoundError as e: 
    print(f"Error: {e}") 
    
with open(path, 'r') as file:
   uni_dt = json.load(file)

In [5]:
# The loaded data has information about university queries. The file contains a list of intents with tags, text(user input), 
# responses, extension, context, entity type and entities. The data is organized as a nested dictionary where 'intents' is the main key.
# Each dictionary value is also a dictionary where the main keys are 'intent', 'text' and 'responses'.

uni_dt['intents'][0:5]

[{'intent': 'greeting',
  'text': ['Hi',
   'How are you?',
   'Is anyone there?',
   'Hello',
   'Good day',
   "What's up",
   'how are ya',
   'heyy',
   'whatsup',
   '??? ??? ??'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'extension': {'function': '', 'entities': False, 'responses': []},
  'context': {'in': '', 'out': 'GreetingUserRequest', 'clear': False},
  'entityType': 'NA',
  'entities': []},
 {'intent': 'goodbye',
  'text': ['cya',
   'see you',
   'bye bye',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Bye',
   'Have a Good day',
   'talk to you later',
   'ttyl',
   'i got to go',
   'gtg'],
  'responses': ['Sad to see you go :(',
   'Talk to you later',
   'Goodbye!',
   'Come back soon'],
  'extension': {'function': '', 'entities': False, 'responses': []},
  'context': {'in': '', 'out': 'LeavingUserRequest', 'clear': False},
  'entityType': 'NA',
  'entities': []},
 {'intent': 'creator',
  'text': ['what is the

In [7]:
#The tags and user input text are reorganized as dataframe for clear representation of tag/intent for each user input.

user_in = []
tag_in = []
for i in range(len(uni_dt['intents'])):
    for k in uni_dt['intents'][i]['text']:
        user_in.append(k)
        tag_in.append(uni_dt['intents'][i]['intent'])
        
uni_data_in = pd.DataFrame({'tags':tag_in, 'user_input':user_in})
uni_data_in

,tags,user_input
0,greeting,Hi
1,greeting,How are you?
2,greeting,Is anyone there?
3,greeting,Hello
4,greeting,Good day
...,...,...
407,Mess Timetable,Bus Timetable
408,Mess Timetable,timetable
409,Mess Timetable,what is the timetable
410,Mess Timetable,what is the bus timetable


In [9]:
#The responses and corresponding tags are also reorganized as dataframe for clear representation of tag/intent for each chatbot response.

tag_out = []
resp = []
for i in range(len(uni_dt['intents'])):
    for k in uni_dt['intents'][i]['responses']:
        resp.append(k)
        tag_out.append(uni_dt['intents'][i]['intent'])

uni_data_out = pd.DataFrame({'tags':tag_out, 'responses':resp})
uni_data_out

,tags,responses
0,greeting,Hello!
1,greeting,Good to see you again!
2,greeting,"Hi there, how can I help?"
3,goodbye,Sad to see you go :(
4,goodbye,Talk to you later
5,goodbye,Goodbye!
6,goodbye,Come back soon
7,creator,College students
8,name,You can call me Mind Reader.
9,name,I'm Mind Reader


In [11]:
# A function is created for preprocessing user text. Stop words have not been excluded as the user content has very short messages with commonly used
# words. Removal of any stopword may remove important text from input patterns. Each user text is converted to lower case. Punctuation marks are removed 
# and the text is subjected to lemmatization for extraction of correct root words.

def preprocess(line):
    line = line.lower()
    line = re.sub(r'[^\w\s]', ' ',line)
    line = line.split()
    word_lem = WordNetLemmatizer()
    line = [word_lem.lemmatize(i) for i in line]
    line = ' '.join(line)
    return line

In [13]:
#The tags are the labelled responses and are encoded into integer labels for training the neural network model.

label_en = LabelEncoder()
labels = label_en.fit_transform(uni_data_in['tags'])
labels

array([15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 14,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 22, 22, 22,
       22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8, 13, 13, 13, 13, 13, 13, 13, 35, 35, 35, 35,
       35, 35, 35, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
       19, 19, 19,  2,  2

In [15]:
# The user input corpus is subjected to preprocessing. The preprocessed text is tokenized to extract a list of unique words
# and the size of vocabulary.

word_set = []
user_sentences = []
for sentence in uni_data_in['user_input']:
    sent = preprocess(sentence)
    word = word_tokenize(sent)
    user_sentences.append(sent)
    word_set.append(word)

vocab = [m for s in word_set for m in s]
vocab = set(vocab)
vocab_size = len(vocab)

In [16]:
# Input data is clean user input and output is the integer label of the corresponding tag.

proc_data_in = pd.DataFrame({'tags':labels, 'user_input':user_sentences})
proc_data_in

,tags,user_input
0,15,hi
1,15,how are you
2,15,is anyone there
3,15,hello
4,15,good day
...,...,...
407,0,bus timetable
408,0,timetable
409,0,what is the timetable
410,0,what is the bus timetable


In [17]:
# The tokenizer function is used to convert the input corpus into sequence vectors.

tokens = Tokenizer(num_words=vocab_size)
tokens.fit_on_texts(proc_data_in['user_input'])
encoded_text = tokens.texts_to_sequences(proc_data_in['user_input'])

In [18]:
#The maximumm length of a user input message is extracted to create sequence vectors of uniform lengths

max_length = 0
for sentence in user_sentences:
    if(max_length < len(sentence.split())):
        max_length = len(sentence.split())

In [19]:
# The sequence vectors are padded to the maximum length.

padded_data = pad_sequences(encoded_text, maxlen=max_length, padding='post', value=0)
padded_data

array([[160,   0,   0, ...,   0,   0,   0],
       [ 13,  10,   5, ...,   0,   0,   0],
       [  1, 161,  26, ...,   0,   0,   0],
       ...,
       [  3,   1,   4, ...,   0,   0,   0],
       [  3,   1,   4, ...,   0,   0,   0],
       [266,   0,   0, ...,   0,   0,   0]])

In [33]:
# Reference used for model building: https://www.kaggle.com/code/uzzivirus/simple-chatbot-using-neural-networks.
# The neural network model has an input of length equal to maximum length. 
# First layer is the embedding layer followed by 3 dense layers.
# The ouput layer has length equal to the number of unique tags. The activation function used is 'Relu' and 'softmax' is used
# for multi class classfication.
warnings.filterwarnings('ignore')

model = Sequential()
model.add(Input(shape=(max_length,)))
model.add(Embedding(input_dim=vocab_size,output_dim=8, input_length=(max_length,)))
model.add(Flatten())
model.add(Dense(units= 64, activation='relu'))
model.add(Dense(units= 64, activation='relu'))
model.add(Dense(units= len(set(labels)), activation='softmax'))

In [35]:
# The optizer used is adam optimiser. Sparse categorical cross entropy is used due to multi class 
# classification and also because the targets are labelled as integers. 

model.compile(loss = "sparse_categorical_crossentropy", optimizer ='adam', metrics=['accuracy'])

In [37]:
model.fit(padded_data, labels, epochs=150, verbose=1)

Epoch 1/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0534 - loss: 3.6598
Epoch 2/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0948 - loss: 3.6390 
Epoch 3/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0942 - loss: 3.6079 
Epoch 4/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1086 - loss: 3.5362 
Epoch 5/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1244 - loss: 3.4928 
Epoch 6/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1250 - loss: 3.4472 
Epoch 7/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1542 - loss: 3.3803
Epoch 8/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1601 - loss: 3.3388 
Epoch 9/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1626 - loss: 3.2308 
Epoch 10/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1789 - loss: 3.1781 
Epoch 11/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1640 - loss: 3.0788 
Epoch 12/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - 

In [39]:
# The argument passed to the chatbot function is the user query. The result returned by the function is the chatbot response.

def chatbot(input_sentence):
    processed_text = preprocess(input_sentence)  #function preprocess defined previously
    
    txt = tokens.texts_to_sequences(processed_text.split())
    coded_txt = [x for y in txt for x in y]
    padded_data = pad_sequences([coded_txt], maxlen=max_length, padding='post', value=0)  # processed text converted into padded sequence
    
    out = (model.predict(padded_data)).argmax()
    
    label_out = label_en.inverse_transform([out]) #convert the predicted label into the corresponding tag
    label_out = ''.join(label_out)
    
    response = uni_data_out[uni_data_out['tags']==label_out]['responses']  #select list of responses corresponding to the tag
    response = random.choice(list(response)) #random selection of one of the respones for the given tag
    return response

In [51]:
input_no = int(input())  #maximum integer input 411

#user input
print('User: ', uni_data_in['user_input'][input_no])

# chatbot response
print('Chatbot: ', chatbot(uni_data_in['user_input'][input_no]))

 178


User:  tell me about events
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Chatbot:  For event detail visit <a target="_blank" href="ADD YOUR FUNCTIONS LINK OR YOUR OWN RESPONSE"> here</a>
